**Importing Dependencies**

In [ ]:
# Gymnasium imports
import gymnasium as gym 
from gymnasium import Env
from gymnasium.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 

# Import helpers
import numpy as np
import random
import os

# Import stable baselines
# Probably not needed
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

**Building the environment**

Description of the environemnt:

- Lorem
- Lorem
- Lorem

Is there a length definition for the class? For how long should the simulation run?

What is the initial state of the environment?

In [ ]:
class sefety_stock_management(Env):
    
    # Define the action and observation space
    def __init__(self):

        # Define action and observation space
        self.action_space = None
        
        self.observation_space = None

        self.state = None

        pass

    # Defining the step function
    def step(self, action):

        # Returns the next state, reward and whether the episode is done

        pass

    def render(self):
        # not needed since visualization is hardly possible for this
        pass

    def reset(self):
        # Reset the state of the environment back to an initial state


        pass
